<a href="https://colab.research.google.com/github/Ritvikg08/Influenza_Sequence_prediction/blob/master/PB_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install Biopython

     |████████████████████████████████| 2.3MB 2.8MB/s 


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime
from Bio import SeqIO

In [ ]:
sequences = [s for s in SeqIO.parse("aligned_sequence.fasta", "fasta")]
metadata = pd.read_csv(r"metadata.csv",sep="\t",parse_dates=["Collection Date"])
type(metadata)

pandas.core.frame.DataFrame

In [ ]:
Accession_list = []
for i in range(len(sequences)):
  Accession_list.append(sequences[i].id[sequences[i].id.index(':')+1:sequences[i].id.index('|')])
print(Accession_list)
Accession_l=[]

Sequence_df = pd.DataFrame()
Sequence_df['Sequence Accession'] = Accession_list
for i in Sequence_df['Sequence Accession']:
  Accession_l.append(i[:8])
print(Accession_l)
Sequence_df['Sequence Accession']=Accession_l
Sequence_df['Sequence'] = [i.seq for i in sequences]
print(Sequence_df)
Main_df = pd.merge(metadata, Sequence_df, on='Sequence Accession')
#Main_df = Main_df.sort_values(by = ['Collection Date'])

#for i in Main_df['Collection Date']:
#  i=i[::-1]
#Main_df=Main_df[Main_df['Collection Date'].apply(lambda x: len(x) > 8)]
#Main_df['Collection Date'] = Main_df['Collection Date'].str[:3]+Main_df['Collection Date'].str[6:]
Main_df = Main_df.drop_duplicates(subset=['Collection Date'], keep='first')
Main_df = Main_df.reset_index(drop=True)
Main_df
#Main_df.to_csv("USA_filtered.csv")
#training_idxs = Main_df[Main_df['Collection Date'].apply(lambda x: int(x[3:]) <= 2018 )]
#training_idxs

In [ ]:
Main_df = Main_df[Main_df['Host Species'] == 'IRD:Human']
Main_df = Main_df[Main_df['Protein Name'] == 'HA']
Main_df = Main_df[Main_df['Subtype'] == 'H1N1']

#Training data
training_idxs = Main_df[Main_df['Collection Date'].apply(lambda x: int(x[3:]) <= 2019 )].index
training_metadata = Main_df[Main_df['Collection Date'].apply(lambda x: int(x[3:]) <= 2019 )]
training_sequences = Main_df[Main_df['Collection Date'].apply(lambda x: int(x[3:]) <= 2019 )]['Sequence']

#Testing data
test_idxs = Main_df[Main_df['Collection Date'].apply(lambda x: int(x[3:]) > 2019 )].index
test_metadata = Main_df[Main_df['Collection Date'].apply(lambda x: int(x[3:]) > 2019 )]
test_sequences = Main_df[Main_df['Collection Date'].apply(lambda x: int(x[3:]) > 2019 )]['Sequence']
print(len(training_idxs))
print(len(test_idxs))
Main_df['Collection Date']
test_metadata

In [ ]:
from collections import Counter
from copy import deepcopy
from sklearn.preprocessing import LabelBinarizer


def encode_array(sequences):

    #To find length of biggest sequence
    total_sequence = len(sequences)
    max_size_sequence = max(Counter([len(s) for s in sequences]).keys())

    #To find common acid that is presant in all amino acids
    common_amino_acid = set()
    for s in sequences:
        common_amino_acid = common_amino_acid.union(set(s))
    
    #To create one-hot encoding of amino acids that is common in all
    one_hot  = LabelBinarizer()
    one_hot.fit(list(common_amino_acid))
    total_common_amino_acid = len(common_amino_acid)

    #To convert sequence into array of character or amino_acids
    padded_sequences = deepcopy(sequences)

    #Padding
    for s in padded_sequences:
        while len(s) < max_size_sequence:s.seq+='-'

    #Sequence after padding
    seq_array = np.chararray(shape=(total_sequence,max_size_sequence),unicode=True)
    for i, seq in enumerate(padded_sequences):
        seq_array[i, :] = list(seq)

    encoded_array = np.zeros(shape=(total_sequence, max_size_sequence*total_common_amino_acid))

    for i in range(max_size_sequence):
        encoded_array[:,i*total_common_amino_acid:(i+1)*total_common_amino_acid] = one_hot.transform(seq_array[:, i])

    return encoded_array

In [ ]:
#sequences=Main_df['Sequence']
print(len(sequences[1]))
sequence_array = encode_array(sequences)
training_array = sequence_array[training_idxs]
test_array = sequence_array[test_idxs]
print(sequence_array.shape)
print(training_array.shape)
print(test_array.shape)

In [ ]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from keras.layers import Input, Dense, Lambda, Dropout
from keras.models import Model, model_from_json
from keras import backend as K
from keras import objectives
from keras.callbacks import EarlyStopping

In [ ]:
# Set up VAE.
with tf.device('/gpu:0'):

    #parameters of model
    intermediate_dim = 1000
    encoding_dim = 3
    latent_dim = encoding_dim
    epsilon_std = 1.0
    nb_epoch = 2000

    x = Input(shape=(training_array.shape[1],))
    z_mean = Dense(latent_dim)(x)
    z_log_var = Dense(latent_dim)(x)

    def sampling(args):
        z_mean, z_log_var = args
        epsilon = K.random_normal(shape=(latent_dim, ), mean=0.,
                                  stddev=epsilon_std)
        return z_mean + K.exp(z_log_var / 2) * epsilon

    #loss function
    def vae_loss(x, x_decoded_mean):
        xent_loss = training_array.shape[1] * objectives.binary_crossentropy(x, x_decoded_mean)
        kl_loss = - 0.5 * K.sum(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)
        return xent_loss + kl_loss

    z = Lambda(sampling, output_shape=(latent_dim,))([z_mean, z_log_var])
    x_decoded_mean = Dense(training_array.shape[1], activation='sigmoid')(z_mean)

    #variational autoencoder
    vae = Model(x, x_decoded_mean)
    vae.compile(optimizer='adam', loss=vae_loss)

    # build a model to project inputs on the latent space
    encoder = Model(x, z_mean)
    encoder_var = Model(x, z_log_var)

    #train test split
    x_train, x_test = train_test_split(training_array)

    early_stopping = EarlyStopping(monitor="val_loss", patience=5)


    # build the decoder
    encoded_input = Input(shape=(encoding_dim,))
    # retrieve the last layer of the autoencoder model
    decoder_layer = vae.layers[-1]
    # create the decoder model
    decoder = Model(inputs=encoded_input, outputs=decoder_layer(encoded_input))


    # Train the VAE to learn weights
    vae.fit(x_train, x_train,
            shuffle=True,
            epochs=nb_epoch,
            validation_data=(x_test, x_test),
            callbacks=[early_stopping],
           )

In [ ]:
encoder.save('encoder')
decoder.save('decoder')

from tensorflow import keras
encoder = keras.models.load_model('encoder')
decoder = keras.models.load_model('decoder')


# Testing of encoder and decoder
test_embeddings_mean = encoder.predict(test_array)
test_binary_mean = decoder.predict(test_embeddings_mean)
for i in range(len(test_binary_mean)):
  test_binary_mean[i] = test_binary_mean[i].round()

In [ ]:
def right_pad(sequences):
    padded_sequences = deepcopy(sequences)
    seq_lengths = compute_seq_lengths(sequences)

    for s in padded_sequences:
        while len(s) < max(seq_lengths.keys()):
            s.seq += '*'
    return padded_sequences


def compute_seq_lengths(sequences):
    seq_lengths = [len(s) for s in sequences]
    seq_lengths = Counter(seq_lengths)
    return seq_lengths


def seq2chararray(sequences):
    padded_sequences = right_pad(sequences)
    seq_lengths = compute_seq_lengths(sequences)
    char_array = np.chararray(shape=(len(sequences), max(seq_lengths.keys())),
                              unicode=True)
    for i, seq in enumerate(padded_sequences):
        char_array[i, :] = list(seq)
    return char_array


def compute_alphabet(sequences):
    alphabet = set()
    for s in sequences:
        alphabet = alphabet.union(set(s))

    return alphabet


def binary2chararrayold(sequences, binary_array):

    alphabet = compute_alphabet(sequences)
    seq_lengths = compute_seq_lengths(sequences)
    seq_array = seq2chararray(sequences)

    lb = LabelBinarizer()
    lb.fit(list(alphabet))

    char_array = np.chararray(shape=(len(binary_array),
                              max(seq_lengths.keys())), unicode=True)

    for i in range(seq_array.shape[1]):
        char_array[:, i] = lb.inverse_transform(binary_array[:, i*len(alphabet):(i+1)*len(alphabet)])

    return char_array

    
alphabet = compute_alphabet(sequences)
seq_lengths = compute_seq_lengths(sequences)
seq_array = seq2chararray(sequences)

def binary2chararray(binary_array):

    lb = LabelBinarizer()
    lb.fit(list(alphabet))

    char_array = np.chararray(shape=(len(binary_array),
                              max(seq_lengths.keys())), unicode=True)

    for i in range(seq_array.shape[1]):
        char_array[:, i] = lb.inverse_transform(binary_array[:, i*len(alphabet):(i+1)*len(alphabet)])

    return char_array

In [ ]:
print(test_array.shape)
test_embeddings_mean = encoder.predict(test_array)
test_binary_mean = decoder.predict(test_embeddings_mean)
result=binary2chararray(test_binary_mean)
print(test_binary_mean.shape)
loop=-1
for seq in sequences[13551:]:
  positive = 0
  negative = 0
  loop+=1
  for i in range(len(seq)):
    if (seq.seq)[i]==result[loop][i]:
      positive+=1
    else:
      negative+=1
  print(positive*100/len(seq))
x_train = encoder.predict(training_array) #The new training data for our model
print(x_train.shape)

In [ ]:
training_embeddings_mean = encoder.predict(training_array)

In [ ]:
def plot_series(time, series, format="-", start=0, end=None, label=None):
    plt.plot(time[start:end], series[start:end], format, label=label)
    plt.xlabel("Time")
    plt.ylabel("Value")
    if label:
        plt.legend(fontsize=14)
    plt.grid(True)
    

In [ ]:
#print(training_array.shape)
#Convert time series into a dataset
def window_dataset(series,window_size,batch_size=16,shuffle_buffer=1000):
  ds=tf.data.Dataset.from_tensor_slices(series)
  ds=ds.window(window_size+1,shift=1,drop_remainder=True)
  ds=ds.flat_map(lambda window: window.batch(window_size+1))
  ds=ds.shuffle(shuffle_buffer)
  ds=ds.map(lambda window: (window[:-1],window[-1]))
  ds=ds.batch(batch_size).prefetch(1)
  count=0
  #for x, y in ds:
  #  print("x =", type(x.numpy())) 
  #  print("y =", y.numpy())
  return ds


In [ ]:
tf.keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)


split_time= 110
full_train=x_train
x_train=x_train[:split_time]
x_valid=full_train[split_time:]
print(x_train[0][0])
print(x_train[1][0])
print(x_train[2][0])


t_data=x_train.transpose() # to get whole train data as one row
v_data=x_valid.transpose()
print(data[0][0:3])


#print(type(train_set))
#for x,y in train_set

window_size=5 #Seasonality taken as 5 timestamps
train_set=window_dataset(x_train,5) # Consider 1 window length = 5 time stamps
valid_set=window_dataset(x_valid,5)



model1_train=t_data[0]

model2_train=t_data[1]

model3_train=t_data[2]


model1_val=v_data[0]

model2_val=v_data[1]

model3_val=v_data[2]

#train_np = np.stack(list(train_set))
#val_np = np.stack(list(valid_set))
#print(type(train_np), train_np.shape)
#print(type(val_np), val_np.shape)
#val_np
#train_np=train_np.reshape(64,5,3)
#len(train_np)
#for x,y in train_set:
  #np.concatenate(x1,x.numpy())
  #print("y =", y.numpy())



In [ ]:
tf.keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

## Prediciton for model1

window_size = 5

model1_train_set = window_dataset(model1_train, window_size)
model1_val_set = window_dataset(model1_val, window_size)
print(model1_train)
model = tf.keras.models.Sequential([
  tf.keras.layers.Dense(10, activation="relu", input_shape=[window_size]),
  tf.keras.layers.Dense(10, activation="relu"),
  tf.keras.layers.Dense(1)
])
optimizer = tf.keras.optimizers.SGD(lr=1e-3, momentum=0.9)
model.compile(loss=tf.keras.losses.Huber(),
              optimizer=optimizer,
              metrics=["mae"])

early_stopping = tf.keras.callbacks.EarlyStopping(patience=10)
model.fit(model1_train_set, epochs=500,validation_data=model1_val_set,callbacks=[early_stopping])



In [ ]:

def model_forecast(model, series, window_size):
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(window_size, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(window_size))
    ds = ds.batch(16).prefetch(1)
    forecast = model.predict(ds)
    return forecast

model1_forecast = model_forecast(model, full_train.transpose()[0][5 - window_size:-1], window_size)[:, 0]
tf.keras.metrics.mean_absolute_error(model1_train, model1_forecast).numpy()

In [ ]:
time_full=np.arange(527)
time_valid=time_full#[527:]
plt.figure(figsize=(10, 6))
plot_series(time_valid, model1_train)
plot_series(time_valid-1, model1_forecast)

In [ ]:
tf.keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

## Prediciton for model2


model2_train_set = window_dataset(model2_train, window_size)
model2_val_set = window_dataset(model2_val, window_size)

model = tf.keras.models.Sequential([
  tf.keras.layers.Dense(10, activation="relu", input_shape=[window_size]),
  tf.keras.layers.Dense(10, activation="relu"),
  tf.keras.layers.Dense(1)
])
optimizer = tf.keras.optimizers.SGD(lr=1e-3, momentum=0.9)
model.compile(loss=tf.keras.losses.Huber(),
              optimizer=optimizer,
              metrics=["mae"])

early_stopping = tf.keras.callbacks.EarlyStopping(patience=10)
model.fit(model2_train_set, epochs=500,validation_data=model2_val_set,callbacks=[early_stopping])


In [ ]:
def model_forecast(model, series, window_size):
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(window_size, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(window_size))
    ds = ds.batch(16).prefetch(1)
    forecast = model.predict(ds)
    return forecast

model2_forecast = model_forecast(model, full_train.transpose()[1][split_time - window_size:-1], window_size)[:, 0]
tf.keras.metrics.mean_absolute_error(model2_val, model2_forecast).numpy()

plt.figure(figsize=(10, 6))
plot_series(time_valid, model2_val)
plot_series(time_valid-1, model2_forecast)

In [ ]:
tf.keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

## Prediciton for model3

model3_train_set = window_dataset(model3_train, window_size)
model3_val_set = window_dataset(model3_val, window_size)

model = tf.keras.models.Sequential([
  tf.keras.layers.Dense(10, activation="relu", input_shape=[window_size]),
  tf.keras.layers.Dense(10, activation="relu"),
  tf.keras.layers.Dense(1)
])
optimizer = tf.keras.optimizers.SGD(lr=1e-3, momentum=0.9)
model.compile(loss=tf.keras.losses.Huber(),
              optimizer=optimizer,
              metrics=["mae"])

early_stopping = tf.keras.callbacks.EarlyStopping(patience=10)
model.fit(model3_train_set, epochs=500,validation_data=model3_val_set,callbacks=[early_stopping])


In [ ]:
def model_forecast(model, series, window_size):
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(window_size, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(window_size))
    ds = ds.batch(16).prefetch(1)
    forecast = model.predict(ds)
    return forecast

model3_forecast = model_forecast(model, full_train.transpose()[2][split_time - window_size:-1], window_size)[:, 0]
tf.keras.metrics.mean_absolute_error(model3_val, model3_forecast).numpy()

plt.figure(figsize=(10, 6))
plot_series(time_valid, model3_val)
plot_series(time_valid-1, model3_forecast)

In [ ]:
#x=np.concatenate(model1_forecast,model2_forecast,model3_forecast)
model1_forecast[14]
x=[model1_forecast[1:4],model2_forecast[1:4],model3_forecast[1:4]]

x=np.array(x)
print(x.shape)
decoded_pred=decoder.predict(x)
decoded_pred.shape

In [ ]:
string_result=binary2chararray(decoded_pred)
print(string_result.shape)
loop=0
for sequence in sequences[528:531]:
  positive = 0
  negative = 0
  for i in range(len(sequence)):
    if (sequence.seq)[i]==string_result[loop][i]:
      positive+=1
    else:
      negative+=1
  loop+=1
  print(positive*100/len(sequence))
  print(negative*100/len(sequence))
  print("\n")

In [ ]:
loop=0
string_result=sequences[1:10]
for sequence in sequences[511:520]:
  positive = 0
  negative = 0
  for i in range(len(sequence)):
    if (sequence.seq)[i]==string_result[loop][i]:
      positive+=1
    else:
      negative+=1
  loop+=1
  print(positive)
  print(negative)
  print("\n")

In [ ]:
################## LSTM WITH CNN OPTIMIZATION ############ 
keras=tf.keras
def seq2seq_window_dataset(series, window_size, batch_size=32,
                           shuffle_buffer=1000):
    series = tf.expand_dims(series, axis=-1)
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(window_size + 1, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(window_size + 1))
    ds = ds.shuffle(shuffle_buffer)
    ds = ds.map(lambda w: (w[:-1], w[1:]))
    return ds.batch(batch_size).prefetch(1)
  

def model_forecast(model, series, window_size):
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(window_size, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(window_size))
    ds = ds.batch(32).prefetch(1)
    forecast = model.predict(ds)
    return forecast

In [ ]:
########FOR SPLITTING INTO TRAINING AND VALIDATION DATA#########
split_time = 1000
time_train = time[:split_time]
x_train = series[:split_time]
time_valid = time[split_time:]
x_valid = series[split_time:]

In [ ]:
###Choose Appropiate Learning Rate ###
plt.semilogx(history.history["lr"], history.history["loss"])
plt.axis([1e-8, 1e-4, 0, 30])

In [ ]:
### PREDICITION MODEL #######

keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)


train_set = seq2seq_window_dataset(x_train, window_size,
                                   batch_size=128)
valid_set = seq2seq_window_dataset(x_valid, window_size,
                                   batch_size=128)

model = keras.models.Sequential([
  keras.layers.Conv1D(filters=32, kernel_size=5,
                      strides=1, padding="causal",
                      activation="relu",
                      input_shape=[None, 1]),
  keras.layers.LSTM(32, return_sequences=True),
  keras.layers.LSTM(32, return_sequences=True),
  keras.layers.Dense(1),
  keras.layers.Lambda(lambda x: x * 20) #####Choose Scaling FACTOR
])
optimizer = keras.optimizers.SGD(lr=1e-5, momentum=0.9)
model.compile(loss=keras.losses.Huber(),
              optimizer=optimizer,
              metrics=["mae"])

model_checkpoint = keras.callbacks.ModelCheckpoint(
    "my_checkpoint.h5", save_best_only=True)
early_stopping = keras.callbacks.EarlyStopping(patience=50)
model.fit(train_set, epochs=500,
          validation_data=valid_set,
          callbacks=[early_stopping, model_checkpoint])